# Test notebook to exercise the Lasair API
### To run this notebook, please [follow the instructions](https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html) or else it won`t work.
The instructions are at https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html

In [1]:
import sys
lsst=True # if you change this restart kernel

if lsst:
    sys.path.append('API_lsst')
    import settings
    endpoint = "https://api.lasair.lsst.ac.uk/api"
    oid = 'diaObjectId'
    raname = 'ra'
    decname = 'decl'
else:
    sys.path.append('API_ztf')
    import settings
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"
    oid = 'objectId'
    raname = 'ramean'
    decname = 'decmean'

In [2]:
import json
from lasair import LasairError, lasair_client as lasair
import settings
L = lasair(settings.API_TOKEN, endpoint=endpoint)
help(lasair)

Help on class lasair_client in module lasair.lasair:

class lasair_client(builtins.object)
 |  lasair_client(token, cache=None, endpoint='https://lasair-ztf.lsst.ac.uk/api', timeout=60.0)
 |
 |  Methods defined here:
 |
 |  __init__(self, token, cache=None, endpoint='https://lasair-ztf.lsst.ac.uk/api', timeout=60.0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  annotate(self, topic, objectId, classification, version='0.1', explanation='', classdict={}, url='')
 |      Send an annotation to Lasair
 |      args:
 |          topic         : the topic for which this user is authenticated
 |          objectId      : the object that this annotation should be attached to
 |          classification: short string for the classification
 |          version       : the version of the annotation engine
 |          explanation   : natural language explanation
 |          classdict     : dictionary with further information
 |          url           : url with further 

This function simply prints the first lines of the given string.
You can replace it with the normal print if you want to see everything.

In [3]:
def print15(s):
    print('\n'.join(s.split('\n')[:15]))

## Cone search

In [4]:
racone  = 54.869
deccone = -27.294
radius = 60 # arcsec
result = L.cone(racone, deccone, radius, requestType='nearest')
print(json.dumps(result, indent=2))

{}


## Database query

In [5]:
selected = 'objects.%s, objects.%s, objects.%s' % (oid, raname, decname)
tables = 'objects,sherlock_classifications'
conditions = 'classification="SN"'
results = L.query(selected, tables, conditions, limit = 8)

for row in results:
    objectId = row[oid]
    ra = row[raname]
    dec = row[decname]
    print(objectId, ra, dec)

# Just use this objectId in subsequent calls
objectId = str(objectId)

169298432598474839 10.498894634371604 -45.1013367036477
169298432603193405 9.521759325592207 -45.56907152470259
169298432612630565 10.815344288798595 -44.292572620711375
169298432613679115 10.902833296883625 -44.06656704594919
169298432615252003 10.897074846367746 -43.78309065688018
169298432615776297 10.564369801933552 -43.97106715675291
169298432752615649 10.606102223449575 -44.03342515148711
169298432872677386 11.030928342063573 -44.889634687110565


## Lightcurves

In [6]:
result = L.object(objectId, lasair_added=False, reliabilityThreshold=0.5)
print15(json.dumps(result, indent=2))

{
  "diaObjectId": "169298432872677386",
  "diaSourcesList": [
    {
      "diaSourceId": 169298432872677386,
      "midpointMjdTai": 60924.331323704115,
      "band": "i",
      "psfFlux": 5388.8662109375,
      "psfFluxErr": 601.2221069335938,
      "reliability": 0.9927553534507751
    },
    {
      "diaSourceId": 169368779559010543,
      "midpointMjdTai": 60940.088714457816,
      "band": "i",


## Sherlock Object

In [7]:
lite=True
result = L.sherlock_object(objectId, lite=lite)
print15(json.dumps(result, indent=2))

{
  "classifications": {
    "169298432872677386": [
      "SN",
      "The transient is possibly associated with <em>10000,96861,2952</em>; an r=21.82 mag galaxy found in the DESI catalogue. It's located 0.85\" N, 0.12\" E from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 21.823,
      "MagErr": 0.019,
      "MagFilter": "r",
      "association_type": "SN",
      "best_distance": null,
      "best_distance_flag": null,


## Sherlock Position

In [8]:
result = L.sherlock_position(ra, dec, lite=lite)
print15(json.dumps(result, indent=2))

{
  "classifications": {
    "query0": [
      "SN",
      "The transient is possibly associated with <em>10000,96861,2952</em>; an r=21.82 mag galaxy found in the DESI catalogue. It's located 0.85\" N, 0.12\" E from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 21.823,
      "MagErr": 0.019,
      "MagFilter": "r",
      "association_type": "SN",
      "best_distance": null,
      "best_distance_flag": null,


## All object data

In [9]:
result = L.object(objectId, lasair_added=True)
print15(json.dumps(result, indent=2))

{
  "diaObjectId": "169298432872677386",
  "lasairData": {
    "nDiaSources": 2,
    "firstDiaSourceMjdTai": null,
    "lastDiaSourceMjdTai": 60940.088714457816,
    "glat": -72.1785,
    "ebv": 0.00789888,
    "rasex": "00:44:7.423",
    "decsex": "-44:53:22.685",
    "ec_lon": 347.3276841325718,
    "ec_lat": -44.54172898796125,
    "g_lon": 307.16787160889373,
    "g_lat": -72.17840908995842,
    "now_mjd": "61058.67",


In [10]:
result = L.object(objectId, lasair_added=False)
print15(json.dumps(result, indent=2))

{
  "diaObjectId": "169298432872677386",
  "diaSourcesList": [
    {
      "diaSourceId": 169298432872677386,
      "midpointMjdTai": 60924.331323704115,
      "band": "i",
      "psfFlux": 5388.8662109375,
      "psfFluxErr": 601.2221069335938,
      "reliability": 0.9927553534507751
    },
    {
      "diaSourceId": 169368779559010543,
      "midpointMjdTai": 60940.088714457816,
      "band": "i",
